# Assay Preparation
This example shows how to generate a worklist for pipetting a simple assay.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy

from robotools import liquidhandling
from robotools import evotools

## Defining labwares
We'll be pipetting an assay for 3x5 samples from a mastermix.

It will also include a calibration curve of another 3x7.

In [2]:
# our assay is pipetted into a 96 well plate
assay = liquidhandling.Labware('AssayPlate', 8, 12, 10, 250)

# we have a trough with 10 ml of mastermix
mastermix = liquidhandling.Labware('MasterMix', 1, 1,
                min_volume=5000, max_volume=100*1000,
                initial_volumes=10000, virtual_rows=8)

# a 96-well plate with 200 µl samples in A01 to C05
samples = liquidhandling.Labware('Samples', 8, 12, 10, 250)
samples.add(samples.wells[:3,:5], 200, label='supernatants')

print(assay)
print(mastermix)
print(samples)

AssayPlate
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
MasterMix
[[10000.]]
Samples
[[200. 200. 200. 200. 200.   0.   0.   0.   0.   0.   0.   0.]
 [200. 200. 200. 200. 200.   0.   0.   0.   0.   0.   0.   0.]
 [200. 200. 200. 200. 200.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]


## Pipetting the assay
Here, we're creating a GWL-worklist that can be given to the EVOware. It will prepare the 96-well assay plate with mastermix and transfer the samples.

The `Worklist` implements almost all commands of EVOware worklists. For example, you can `decontaminate`, `wash`, `set_ditis`, or use `aspirate_well`/`dispense_well` to customize individual steps.

In [3]:
with evotools.Worklist() as wl:
    wl.wash()
    # put buffer into all wells
    wl.transfer(
        mastermix, mastermix.wells[:3,[0]*5],
        assay, assay.wells[:3,:5],
        50,
        label='initial buffer'
    )
    
    # add samples column by column, with wash steps inbetween
    for c in range(5):
        wl.wash()
        wl.transfer(
            samples, samples.wells[:3,c],
            assay, assay.wells[:3,c],
            10,
            label=f'add samples from column {c+1}'
        )
    

Our three labwares have been modified by the pipetting steps. We can inspect the history of the 96-well plate to check if we selected the right wells & volumes:

In [4]:
print(assay.report)

AssayPlate
initial
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

initial buffer
[[50. 50. 50. 50. 50.  0.  0.  0.  0.  0.  0.  0.]
 [50. 50. 50. 50. 50.  0.  0.  0.  0.  0.  0.  0.]
 [50. 50. 50. 50. 50.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

add samples from column 1
[[60. 50. 50. 50. 50.  0.  0.  0.  0.  0.  0.  0.]
 [60. 50. 50. 50. 50.  0.  0.  0.  0.  0.  0.  0.]
 [60. 50. 50. 50. 50.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0

## Inspecting the result
Relevant for the robot is just the worklist, which is merely a list of all individual aspirate/dispense/wash/... steps.

Note that it includes the `label`s as comments.

In [5]:
wl

W1;
C;initial buffer
A;MasterMix;;;1;;50.00;;;;
D;AssayPlate;;;1;;50.00;;;;
A;MasterMix;;;2;;50.00;;;;
D;AssayPlate;;;2;;50.00;;;;
A;MasterMix;;;3;;50.00;;;;
D;AssayPlate;;;3;;50.00;;;;
A;MasterMix;;;1;;50.00;;;;
D;AssayPlate;;;9;;50.00;;;;
A;MasterMix;;;2;;50.00;;;;
D;AssayPlate;;;10;;50.00;;;;
A;MasterMix;;;3;;50.00;;;;
D;AssayPlate;;;11;;50.00;;;;
A;MasterMix;;;1;;50.00;;;;
D;AssayPlate;;;17;;50.00;;;;
A;MasterMix;;;2;;50.00;;;;
D;AssayPlate;;;18;;50.00;;;;
A;MasterMix;;;3;;50.00;;;;
D;AssayPlate;;;19;;50.00;;;;
A;MasterMix;;;1;;50.00;;;;
D;AssayPlate;;;25;;50.00;;;;
A;MasterMix;;;2;;50.00;;;;
D;AssayPlate;;;26;;50.00;;;;
A;MasterMix;;;3;;50.00;;;;
D;AssayPlate;;;27;;50.00;;;;
A;MasterMix;;;1;;50.00;;;;
D;AssayPlate;;;33;;50.00;;;;
A;MasterMix;;;2;;50.00;;;;
D;AssayPlate;;;34;;50.00;;;;
A;MasterMix;;;3;;50.00;;;;
D;AssayPlate;;;35;;50.00;;;;
W1;
C;add samples from column 1
A;Samples;;;1;;10.00;;;;
D;AssayPlate;;;1;;10.00;;;;
A;Samples;;;2;;10.00;;;;
D;AssayPlate;;;2;;10.00;;;;
A;Sam